## Demystifying the mathematics behind PCA

We all know PCA and we all love PCA. Our friend that helps us deal with the curse of dimensionality. Everyone has probably used [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html). I thought I knew PCA.

Until I was asked to explain the mathematics behind PCA in an interview and all I could murmur was that it somehow maximizes the variance of the along the new dimensions. The interviewer was even kind enough to throw me a hint about projections. To my chagrin, I couldn't figure it out even then and had to admit I was stumped. To rub salt in my wounds, I knew that I was taught the mathematics behind PCA during the first year of my Master's course. So here's a post to make sure that doesn't happen to you, dear reader, and hopefully me as well :smile:.  

WARNING: This post will be long and mathematical as the title suggests.

## The basics:

PCA stands for Principal Component Analysis. If you have multidimensional data that's giving you a hard time when you try to train a model on it, PCA could be the answer. You could also visualize high dimensional data using PCA, which is done often in [NLP](https://necromuralist.github.io/Neurotic-Networking/posts/nlp/pca-dimensionality-reduction-and-word-vectors/index.html).

In [1]:
# there's no understanding without doing, so let's write some code
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_wine
np.random.seed(420)

<sub><sup>(hidden latex commands)</sup></sub>
$\renewcommand{\Cov}{\mathrm{Cov}}$
$\renewcommand{\var}{\mathrm{var}}$

We have a set of data, that we will call $x$. $x$ is $D$ dimensional, hence $x \in \mathbb{R}^D$ (just a fancy way of saying that our points are real numbers in D dimensions). Our goal to represent $x$ in less than $D$ dimensions. This is by no means going to be a perfect transition. Say we project down from $D$ dimensions to $M$ dimensions. By definition, $M<D$.

In [2]:
# say D=10, i.e we have 10 dimensional data.
x = np.random.rand(10,10)
# for the geeks amongst us this is from a continuous uniform distrbution from [0, 1)
x.shape
# so we have 10 data points each of which have 10 dimensions

(10, 10)

The typical approach is to maximize the variance of the $M$ dimensional data in such a way that it captures as much of the variance of the $D$ dimensional data as possible. Let's call $z$ the co-ordinates of the projection of $x$ into the lower M dimensional space.

[Harold Hotelling](https://en.wikipedia.org/wiki/Harold_Hotelling) is credited with coming up with the idea of minimizing the variance in 1933. Mr.Hotelling was quite the madlad, since he also came up with the T-square distribution amongst other laws, lemmas and rules. He wrote presciently in his original paper, 90 years ago: 
>"It is natural to ask whether some more fundamental set of independent variables exists, perhaps fewer in number than the $x$'s, which determine the values the $x$'s will take."

## Gentle math:

Let's introduce the mean. This is our "average" friend.
$$\bar{x}=\frac{1}{n}\sum_{i}^{n}x_i$$

In [3]:
# in numpy if we don't define the axis, it will compute the mean of all the 100 elements that we entered
x.mean()

0.45763203983322837

In [4]:
# since we're dealing with components, we are more interested in the columwise or featurewise mean
x.mean(axis=0)

array([0.58607683, 0.35993049, 0.4649175 , 0.53540096, 0.51442973,
       0.50757007, 0.42450736, 0.25607604, 0.50758928, 0.41982214])

In [5]:
# if you're paranoid and you'd like to check the values, here you go:
for i in range(0,10):
    print(x[:,i].sum()/10)

0.5860768282330243
0.359930493813573
0.46491749778618263
0.5354009642518928
0.5144297337485908
0.5075700667940669
0.42450736094776953
0.2560760356268859
0.5075892818724383
0.4198221352578594


#### Standard deviation:

The standard deviation is just how much each point deviates from the mean. It measures how spread out our data is.

$$\sigma=\sqrt{\frac{1}{n}\sum_{i}^{n}(x_i - \bar{x})^2}$$

In [6]:
x.std(axis=0)

array([0.29150845, 0.25559572, 0.30291139, 0.24915131, 0.29970886,
       0.26688689, 0.24686484, 0.25656166, 0.27335956, 0.275878  ])

In [7]:
# say we have some different data:
y = np.array([1]*6)
z = np.array([0]*3+[1]*3)
w = np.arange(1,7)
arrays = [y, z, w]

In [8]:
# we can check their means:
for _ in arrays:
    print(_.mean())

1.0
0.5
3.5


In [9]:
# we can also check their standard deviation:
for _ in arrays:
    print(_.std())

0.0
0.5
1.707825127659933


In [10]:
# paranoia tax
for _ in arrays:
    current_mean = _.mean()
    print(np.sqrt(np.sum((_ - current_mean)**2)/len(_)))

0.0
0.5
1.707825127659933


This makes sense when you consider that our array y had only one value and has a standard deviation of 0. Our array z was slightly better spread out, but still it only had 2 unique values each occuring thrice. Our array w was the most numerically diverse of y, z and w, and hence it had the highest standard deviation.

#### Variance:

The variance is simply the square of the standard deviation.

$\renewcommand{\var}{\mathrm{var}}$
$$ \var(x) = \sigma^2=\frac{1}{n}\sum_{i}^{n}(x_i - \bar{x})^2$$

In [11]:
x.var(axis=0)

array([0.08497718, 0.06532917, 0.09175531, 0.06207638, 0.0898254 ,
       0.07122861, 0.06094225, 0.06582388, 0.07472545, 0.07610867])

In [12]:
# consider the 1d line data again
for _ in arrays:
    print(_.var())

0.0
0.25
2.9166666666666665


All the measures we looked at so far were dealing with only one dimension. Notice that while we have just one number to describe the mean, $\sigma$ and $\sigma^2$ for our one liner data (y,z,w), since they contain only one dimension; we had to specify an axis for $x$, since $x$ is a collection of 10 data points with 10 dimensions each. Also notice that when dealing with $x$, all 3 measures had 10 values, that is as many values as the number of dimensions in $x$.

#### Covariance:

To figure out the interactions between the dimensions and how they vary depending on each other, we introduce the aptly named covariance term. The covariance is always measured between two dimensions. Notice the similarities between the variance and the covariance below:

$\renewcommand{\var}{\mathrm{var}}$
$$ \var(x) \frac{1}{n}\sum_{i}^{n}(x_i - \bar{x})(x_i - \bar{x})$$

$\renewcommand{\Cov}{\mathrm{Cov}}$
$$ \Cov(x,y) = \frac{1}{n}\sum_{i}^{n}(x_i - \bar{x})(y_i - \bar{y})$$

It might be nice to see the covariance of variable on a real dataset rather than the random data we have here:

In [22]:
# toy data from the wine sklearn dataset
from sklearn.datasets import load_wine
data = load_wine()["data"]
data.shape
# we have 13 features

(178, 13)

Remember that covariance can only capture the variance between 2 variables. Additionally, we also know that the covarince of a feature with itself is the variance. Perhaps $\Cov(a,b) \neq \Cov(b,a)$. So, as far as a count of covariances goes, in this case with 13 features, we will have a 13x13 matrix, because of the reasons we just mentioned.

In [24]:
# features are in columns, hence rowvar is false
np.cov(data, rowvar=False).shape

(13, 13)

In [28]:
np.cov(data, rowvar=False)

array([[ 6.59062328e-01,  8.56113090e-02,  4.71151590e-02,
        -8.41092903e-01,  3.13987812e+00,  1.46887218e-01,
         1.92033222e-01, -1.57542595e-02,  6.35175205e-02,
         1.02828254e+00, -1.33134432e-02,  4.16978226e-02,
         1.64567185e+02],
       [ 8.56113090e-02,  1.24801540e+00,  5.02770393e-02,
         1.07633171e+00, -8.70779534e-01, -2.34337723e-01,
        -4.58630366e-01,  4.07333619e-02, -1.41146982e-01,
         6.44838183e-01, -1.43325638e-01, -2.92447483e-01,
        -6.75488666e+01],
       [ 4.71151590e-02,  5.02770393e-02,  7.52646353e-02,
         4.06208278e-01,  1.12293658e+00,  2.21455913e-02,
         3.15347299e-02,  6.35847140e-03,  1.51557799e-03,
         1.64654327e-01, -4.68215451e-03,  7.61835841e-04,
         1.93197391e+01],
       [-8.41092903e-01,  1.07633171e+00,  4.06208278e-01,
         1.11526862e+01, -3.97476036e+00, -6.71149146e-01,
        -1.17208281e+00,  1.50421856e-01, -3.77176220e-01,
         1.45024186e-01, -2.09118054e

Positive covariance values between two features indicate that as one feature increases the other also increases. Negative values indicate an increase-decrease relation.

## Starting to get serious math:

    When trying to do PCA, we can follow one of two approaches:
        1. Minimize the reconstruction error
        2. Maximize the variance

## References:
    1. Slides by Pascal Vincet and explanations by Ioannis Mitliagkas in the course IFT630 - Fundamentals of Machine Learning taught at the University of Montreal.